## Timeseries clustering

Time series clustering is to partition time series data into groups based on similarity or distance, so that time series in the same cluster are similar.

Methodology followed:
* Use Variational Recurrent AutoEncoder (VRAE) for dimensionality reduction of the timeseries
* To visualize the clusters, PCA and t-sne are used

Paper:
https://arxiv.org/pdf/1412.6581.pdf

#### Contents

0. [Load data and preprocess](#Load-data-and-preprocess)
1. [Initialize VRAE object](#Initialize-VRAE-object)
2. [Fit the model onto dataset](#Fit-the-model-onto-dataset)
3. [Transform the input timeseries to encoded latent vectors](#Transform-the-input-timeseries-to-encoded-latent-vectors)
4. [Save the model to be fetched later](#Save-the-model-to-be-fetched-later)
5. [Visualize using PCA and tSNE](#Visualize-using-PCA-and-tSNE)

In [ ]:
# from IPython.display import HTML
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
# if (code_show){
# $('div.input').hide();
# } else {
# $('div.input').show();
# }
# code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Import required modules

In [ ]:
import copy
import numpy as np
import pandas as pd

from vrae.vrae import VRAE
from vrae.utils import *
import torch
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, confusion_matrix

from tensorflow.keras.utils import to_categorical

In [ ]:
gpu_id = 2

if gpu_id>=0:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    cuda_id = "cuda:" + str(0)  # cuda:2

device = torch.device(cuda_id if torch.cuda.is_available() else "cpu")
print("Device:", device)
if (torch.cuda.is_available()):
    torch.cuda.set_device(cuda_id)
    print("Current GPU ID:", torch.cuda.current_device())

### Input parameters

In [ ]:
dload = './model_dir' #download directory

### Hyper parameters

In [ ]:
hidden_size = 90
hidden_layer_depth = 1
latent_length = 20
batch_size = 64
learning_rate = 0.0005
n_epochs = 600
dropout_rate = 0
optimizer = 'Adam' # options: ADAM, SGD
cuda = True # options: True, False
print_every=30
clip = False # options: True, False
max_grad_norm=5
loss = 'MSELoss' # options: SmoothL1Loss, MSELoss
block = 'LSTM' # options: LSTM, GRU

corr = 'Both' # options: Gaussian, ZeroMask, ConsecutiveZeros, Both
sigma = 0.35 # 0.1
if corr == 'ConsecutiveZeros':
    sigma = [50, 80] # [lambda_corr, lambda_norm]
if corr == 'Both':
    sigma = [0.1, 30, 80]

seed = 0

In [ ]:
# random.seed(args.random_seed)
np.random.seed(seed)
# generator = torch.Generator()
# generator.manual_seed(seed)
torch.manual_seed(seed)

### Load data and preprocess

In [ ]:
window_len = 512 # 512
stride_len = 20 # 100
act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
# act_list = [1, 2]
act_labels_txt = ['lay', 'sit', 'std', 'wlk', 'run', 'cyc', 'nord', 'ups', 'dws', 'vac', 'iron', 'rop']

In [ ]:
X=[]
user_labels=[]
act_labels=[]

# columns for IMU data
# 4-20 IMU hand
# 21-37 IMU chest
# 38-54 IMU ankle
# 2-4 3D-acceleration data (ms-2), scale: ±16g, resolution: 13-bit
# 8-10 3D-gyroscope data (rad/s)
# 11-13 3D-magnetometer data (μT)
imu_locs = [4,5,6, 10,11,12, 13,14,15, 
            21,22,23, 27,28,29, 30,31,32, 
            38,39,40, 44,45,46, 47,48,49
           ] 

# scaler = StandardScaler()

for uid in np.arange(1,10):
    path = '../../PAMAP2_Dataset/Protocol/subject10' + str(uid) + '.dat'
    df = pd.read_table(path, sep=' ', header=None)
    act_imu_filter = df.iloc[:, imu_locs] 


    for act_id in range(len(act_list)):
        act_filter =  act_imu_filter[df.iloc[:, 1] == act_list[act_id]]
        act_data = act_filter.to_numpy()
        act_data = np.transpose(act_data)

        # sliding window segmentation
        start_idx = 0
        while start_idx + window_len < act_data.shape[1]:
            window_data = act_data[:, start_idx:start_idx+window_len]
            downsamp_data = window_data[:, ::3] # downsample from 100hz to 33.3hz
            downsamp_data = np.nan_to_num(downsamp_data) # remove nan
            # downsamp_data = np.transpose(downsamp_data) # dim: seq_len, feature_len

            X.append(downsamp_data)
            user_labels.append(uid)
            act_labels.append(act_id)
            start_idx = start_idx + stride_len
            
X = np.array(X).astype('float32')

In [ ]:
normalized_X = np.zeros_like(X)
for ch_id in range(X.shape[1]):
    ch_data = X[:, ch_id, :]
    scaler = MinMaxScaler()
    ch_data = scaler.fit_transform(ch_data)
    normalized_X[:, ch_id, :] = ch_data
X = np.transpose(normalized_X, (0, 2, 1)) # num_samples, sequence_length, feature_length

In [ ]:
# X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]) # convert list to numpy array
act_labels = np.array(act_labels).astype('float32')
act_labels = act_labels.reshape(act_labels.shape[0],1)
act_labels = to_categorical(act_labels, num_classes=len(act_list))

In [ ]:
print(X.shape)
print(act_labels.shape)

In [ ]:
dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(act_labels))

# Train/Test dataset split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainLoader = torch.utils.data.DataLoader(train_dataset,
    batch_size=batch_size, shuffle=True) 

testLoader = torch.utils.data.DataLoader(test_dataset,
    batch_size=batch_size, shuffle=False)

In [ ]:
sequence_length = X.shape[1]
number_of_features = X.shape[2]

In [ ]:
print(sequence_length)
print(number_of_features)

### Initialize VRAE object

VRAE inherits from `sklearn.base.BaseEstimator` and overrides `fit`, `transform` and `fit_transform` functions, similar to sklearn modules

In [ ]:
# learning_rate = 5*1e-4
vrae = VRAE(sequence_length=sequence_length,
            number_of_features = number_of_features,
            corr = corr,
            sigma = sigma,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate,
            optimizer = optimizer, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            loss = loss,
            block = block,
            dload = dload)

### Fit the model onto dataset

In [ ]:
vrae.fit(train_dataset)

# If the model has to be saved, with the learnt parameters use:
# model saving during training to be implemented, use manual saving in the next block
# vrae.fit(dataset, save = True)

### Save the model to be fetched later

In [ ]:
# Save VRAE model
model_name = 'vrae_' + corr
for v in sigma:
    model_name += '_' + str(v)
model_name += '.pth'

vrae.save(model_name)

## Evaluation

In [ ]:
# Load VRAE model from pth file
model_name = 'vrae_' + corr
for v in sigma:
    model_name += '_' + str(v)
model_name += '.pth'

vrae.load(model_name)

In [ ]:
# Prepare testdata set, drop the last incomplete batch
test_x = torch.zeros(len(test_dataset)-35, X.shape[1], X.shape[2])
test_labels = torch.zeros(len(test_dataset)-35, len(act_list))
for test_id in range(len(test_dataset)-35):
    test_labels[test_id] = test_dataset[test_id][1]
    test_x[test_id] = test_dataset[test_id][0]

In [ ]:
print(test_x.shape)

### Corrupt Dataset

In [ ]:
corr_test_x = vrae.encoder.corrupt(test_x.permute(1,0,2)) # Corruption process is different than used in reconstruction
corr_test_x = corr_test_x.permute(1,0,2) # N*171*27

In [ ]:
print(corr_test_x.shape)

### Reconstruct testset (synthesize the entire data)

In [ ]:
# drop_last_test_dataset = TensorDataset(test_x.permute(0, 2, 1))
# corr_test_dataset = TensorDataset(corr_test_x.permute(0,2,1))
corr_test_dataset = TensorDataset(corr_test_x)

In [ ]:
np_recon_test = vrae.reconstruct(corr_test_dataset)
recon_test = torch.from_numpy(np_recon_test).permute(1, 0, 2)

### Reconstruct filling testset (fills missing values only, not valid for noisy data)

In [ ]:
recon_fill_test = copy.deepcopy(corr_test_x.cpu().numpy())
np.copyto(recon_fill_test, recon_test.cpu().numpy(), where = recon_fill_test==0)
recon_fill_test = torch.from_numpy(recon_fill_test)

### Mean-Filling

In [ ]:
mean_fill_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
for i in range(mean_fill_test.shape[0]):
    for j in range(mean_fill_test.shape[2]):
        if np.count_nonzero(mean_fill_test[i,:,j]) == 0: # when all data points in this channel are missing
            ch_mean = 0
        else:
            ch_mean = np.sum(mean_fill_test[i,:,j]) / np.count_nonzero(mean_fill_test[i,:,j])
        mean_fill_test[i, mean_fill_test[i,:,j] == 0, j] = ch_mean
mean_fill_test = torch.from_numpy(mean_fill_test)

### Linear Interpolation

In [ ]:
linear_interp_test = copy.deepcopy(corr_test_x).detach().cpu().numpy()
for i in range(linear_interp_test.shape[0]):
    for j in range(linear_interp_test.shape[2]):
        if np.count_nonzero(linear_interp_test[i,:,j]) == 0: # when all data points in this channel are missing
            linear_interp_test[i, :, j] = 0.0
        else:
            idxs = np.arange(linear_interp_test.shape[1]) # indexes of all the samples
            zero_filter = linear_interp_test[i,:,j] == 0 # index filter for zero values
            zero_idxs = idxs[zero_filter] # indexes for zero values
            non_zero_idxs = idxs[~zero_filter] # xp, indexes for non-zero values
            non_zero_vals = linear_interp_test[i, ~zero_filter, j] # fp, non-zero values
            interp_vals = np.interp(zero_idxs, non_zero_idxs, non_zero_vals) # interpolated values
            linear_interp_test[i,zero_idxs,j] = interp_vals # fill interpolated values to the corrupted signal
linear_interp_test = torch.from_numpy(linear_interp_test)

In [ ]:
corr_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().numpy(), corr_test_x.reshape(corr_test_x.shape[0],-1).cpu().numpy(), squared=False)
print('Corr RMSE:\n' + str(corr_rms))

In [ ]:
recon_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().numpy(), recon_test.reshape(recon_test.shape[0],-1).cpu().numpy(), squared=False)
print('Recon RMSE:\n' + str(recon_rms))

In [ ]:
recon_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1).cpu().numpy(), recon_fill_test.reshape(recon_fill_test.shape[0],-1).cpu().numpy(), squared=False)
print('Recon Fill RMSE:\n' + str(recon_fill_rms))

In [ ]:
mean_fill_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1), mean_fill_test.reshape(mean_fill_test.shape[0],-1).cpu().numpy(), squared=False)
print('Mean Fill RMSE:\n' + str(mean_fill_rms))

In [ ]:
linear_interp_rms = mean_squared_error(test_x.reshape(test_x.shape[0],-1), linear_interp_test.reshape(linear_interp_test.shape[0],-1).cpu().numpy(), squared=False)
print('Linear Interpolation RMSE:\n' + str(linear_interp_rms))

### Plot Data Sample

In [ ]:
data_id = 0

plt.imshow(test_x[data_id])
plt.title('Raw Data')
plt.show()

plt.imshow(corr_test_x[data_id])
plt.title('Corrupted')
plt.show()

plt.imshow(recon_test[data_id])
plt.title('Reconstructed')
plt.show()

plt.imshow(recon_fill_test[data_id])
plt.title('Recon Fill')
plt.show()

plt.imshow(mean_fill_test[data_id])
plt.title('Mean Fill')
plt.show()

plt.imshow(linear_interp_test[data_id])
plt.title('Linear Interpolation')
plt.show()

### Import HAR model

In [ ]:
from eval_har import *

In [ ]:
eval_har = get_eval_model(len(act_list), model_path='pamap2_cnn.pt').to(device)

### Evaluate HAR accuracy for raw data, corrupted data, and reconstructed data

In [ ]:
# Convert from N*171*27 into N*27*171
test_x = test_x.permute(0,2,1)
corr_test_x = corr_test_x.permute(0,2,1)
recon_test = recon_test.permute(0,2,1)
recon_fill_test = recon_fill_test.permute(0,2,1)
mean_fill_test = mean_fill_test.permute(0,2,1)
linear_interp_test = linear_interp_test.permute(0,2,1)

In [ ]:
# Extend one dummy dimension for CNN
raw_test_dataset = TensorDataset(test_x.reshape(test_x.shape[0], 1, test_x.shape[1], test_x.shape[2]), test_labels)
corr_test_dataset = TensorDataset(corr_test_x.reshape(corr_test_x.shape[0], 1, corr_test_x.shape[1], corr_test_x.shape[2]), test_labels)
recon_test_dataset = TensorDataset(recon_test.reshape(recon_test.shape[0], 1, recon_test.shape[1], recon_test.shape[2]), test_labels)
recon_fill_test_dataset = TensorDataset(recon_fill_test.reshape(recon_fill_test.shape[0], 1, recon_fill_test.shape[1], recon_fill_test.shape[2]), test_labels)
mean_fill_test_dataset = TensorDataset(mean_fill_test.reshape(mean_fill_test.shape[0], 1, mean_fill_test.shape[1], mean_fill_test.shape[2]), test_labels)
linear_interp_test_dataset = TensorDataset(linear_interp_test.reshape(linear_interp_test.shape[0], 1, linear_interp_test.shape[1], linear_interp_test.shape[2]), test_labels)

In [ ]:
def evaluate_har(test_loader):
    correct = 0
    total = 0
    total_true = []
    total_pred = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)       
            # calculate outputs by running images through the network
            outputs = eval_har(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            # predicted = torch.argmax(outputs.data.cpu(), axis=1)
            total += labels.size(0)
            correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
            
            total_pred = total_pred + predicted.cpu().numpy().tolist()
            total_true = total_true + (torch.argmax(labels, dim=1).cpu().numpy().tolist())
            
    print(f'Test Accuracy: {100.0 * correct / total} %')
    
    print(" | ".join(act_labels_txt))
    conf_mat = confusion_matrix(y_true = total_true, y_pred = total_pred)
    conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
    print(np.array(conf_mat).round(3) * 100)
    
    return

In [ ]:
print("Raw testset:")
raw_test_loader = torch.utils.data.DataLoader(raw_test_dataset,
    batch_size=batch_size, shuffle=False)

evaluate_har(raw_test_loader)

In [ ]:
print("Corrupted testset:")
corr_test_loader = torch.utils.data.DataLoader(corr_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(corr_test_loader)

In [ ]:
print("Reconstructed testset:")
recon_test_loader = torch.utils.data.DataLoader(recon_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(recon_test_loader)

In [ ]:
print("Recon Fill testset:")
recon_fill_test_loader = torch.utils.data.DataLoader(recon_fill_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(recon_fill_test_loader)

In [ ]:
print("Mean Fill testset:")
mean_fill_test_loader = torch.utils.data.DataLoader(mean_fill_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(mean_fill_test_loader)

In [ ]:
print("Linear Interpolation testset:")
linear_interp_test_loader = torch.utils.data.DataLoader(linear_interp_test_dataset,
    batch_size=batch_size, shuffle=False)
evaluate_har(linear_interp_test_loader)

### Plot test data for each activity

In [ ]:
test_labels_np = torch.argmax(test_labels, dim=1).detach().cpu().numpy() # labels of testset, not one-hot
idx_list = np.arange(len(test_labels_np)) # list for indexes of all testset
rand_act_idxs = [] 
# randomly select a data sample for each activity
for act_id in range(len(act_list)):
    act_filter = test_labels_np[:] == act_id # f
    act_idx_list = idx_list[act_filter]
    rand_act_id = np.random.choice(act_list)
    rand_act_idxs.append(rand_act_id)

In [ ]:
fig = plt.figure(figsize=(50, 25))

for act_id in range(len(rand_act_idxs)):
    ax1 = fig.add_subplot(12,5,1+5*act_id)
    ax1.imshow(test_x[rand_act_idxs[act_id]], vmin=0, vmax=1)
    ax1.set_title('Raw Data - ' + act_labels_txt[act_id])
    
    # Uncomment for plotting the difference between the target case and the raw data
#     ax2 = fig.add_subplot(12,5,2+5*act_id)
#     ax2.imshow(test_x[rand_act_idxs[act_id]] - corr_test_x[rand_act_idxs[act_id]], vmin=0, vmax=1)
#     ax2.set_title('Corrupted - ' + act_labels_txt[act_id])
    
#     ax3 = fig.add_subplot(12,5,3+5*act_id)
#     ax3.imshow(test_x[rand_act_idxs[act_id]] - recon_test[rand_act_idxs[act_id]], vmin=0, vmax=1)
#     ax3.set_title('Reconstructed - ' + act_labels_txt[act_id])
    
#     ax4 = fig.add_subplot(12,5,4+5*act_id)
#     ax4.imshow(test_x[rand_act_idxs[act_id]] - recon_fill_test[rand_act_idxs[act_id]], vmin=0, vmax=1)
#     ax4.set_title('Recon Fill - ' + act_labels_txt[act_id])

#     ax5 = fig.add_subplot(12,5,5+5*act_id)
#     ax5.imshow(test_x[rand_act_idxs[act_id]] - mean_fill_test[rand_act_idxs[act_id]], vmin=0, vmax=1)
#     ax5.set_title('Mean Fill - ' + act_labels_txt[act_id])

    # Plot the target case
    ax2 = fig.add_subplot(12,5,2+5*act_id)
    ax2.imshow(corr_test_x[rand_act_idxs[act_id]], vmin=0, vmax=1)
    ax2.set_title('Corrupted - ' + act_labels_txt[act_id])
    
    ax3 = fig.add_subplot(12,5,3+5*act_id)
    ax3.imshow(recon_test[rand_act_idxs[act_id]], vmin=0, vmax=1)
    ax3.set_title('Reconstructed - ' + act_labels_txt[act_id])
    
    ax4 = fig.add_subplot(12,5,4+5*act_id)
    ax4.imshow(recon_fill_test[rand_act_idxs[act_id]], vmin=0, vmax=1)
    ax4.set_title('Recon Fill - ' + act_labels_txt[act_id])

    ax5 = fig.add_subplot(12,5,5+5*act_id)
    ax5.imshow(mean_fill_test[rand_act_idxs[act_id]], vmin=0, vmax=1)
    ax5.set_title('Mean Fill - ' + act_labels_txt[act_id])

### Transform the input timeseries to encoded latent vectors

In [ ]:
# z_run = vrae.transform(test_dataset)

# #If the latent vectors have to be saved, pass the parameter `save`
# # z_run = vrae.transform(dataset, save = True)

### Visualize using PCA and tSNE

In [ ]:
# plot_clustering(z_run, y_val, engine='matplotlib', download = False)

# # If plotly to be used as rendering engine, uncomment below line
# #plot_clustering(z_run, y_val, engine='plotly', download = False)